#Descripción

En Homie.mx se busca conectar a buenos propietarios con buenos inquilinos. Para esto, se busca ayudar a las personas a encontrar un hogar al tiempo que generamos una fuente de ingresos para los propietarios. 

Para Homie.mx es muy importante la experiencia de sus usuarios, por lo que es de suma importancia que las propiedades esten bien segmentadas y que su busqueda sea lo más sencilla posibls.


#Exploración y Limpieza de Datos

* Base de scrapps: muestra aleatoria de toda la data generada con web scrapers. Algunos features incluyen tamaño del inmueble, si cuenta con estacionamiento, número de cuartos y baños, precio de renta, etc.
* Base colonias de la CdMx: tomado originalmente de una API pública y que cuenta con features como la geometría y localización del inmueble.

Ambas bases de datos provienen en formato .csv.

**Se importa la primera base de datos para fase exploratoria inicial**. Para esto se hará uso de la libreriía Pandas, por la facilidad que provee para la importación y manipulación de los datos

In [ ]:
import pandas as pd
 

In [ ]:
rentas = pd.read_csv("rents.csv", sep = ",")
rentas.head(10)

,Unnamed: 0,rooms,bathrooms,parking,precio,latitude,longitude,date_in
0,0,3,1.0,True,1800.0,19.362000,99.225200,2020-06-30 03:40:29.302309
1,1,2,1.0,False,3200.0,19.343523,99.051770,2020-09-07 21:58:26.147000
2,2,dos,2.0,True,27000.0,19.364412,99.286790,2020-10-03 09:32:28.608233
3,3,3,2.0,True,8800.0,19.364382,99.286820,2020-12-12 08:14:50.260666
4,4,dos,3.0,True,36000.0,19.225096,98.199100,2020-12-13 08:31:08.708283
5,5,3,2.0,True,30000.0,19.373072,99.253334,2020-10-11 01:46:41.486535
6,6,3,2.0,False,23000.0,19.386393,99.225560,2020-12-02 14:31:29.019640
7,7,dos,2.0,True,14500.0,19.384823,99.193930,2020-10-24 23:53:27.131001
8,8,3.0,1.0,True,9500.0,19.368000,99.219000,2020-08-24 18:19:48.119703
9,9,uno,2.0,True,19000.0,19.359640,99.192810,2020-08-09 11:09:49.880061


En un primer vistazo rápido es notable que exiten varios tipos de datos: enteros, strings, flotantes y timestamps. Es importante ahora verificar la consistencia de estos datos, es decir, revisar la existencia de datos nulos o inconsistentes. 
Una primer columna para esta verificación es "rooms", ya que a simple vista es notable que cuenta con mas de 1 tipo de dato.


In [ ]:
rentas.describe()

,Unnamed: 0,bathrooms,precio,latitude,longitude
count,2000.000000,1933.000000,2.000000e+03,2000.000000,2000.000000
mean,999.500000,1.881531,1.127989e+08,19.547428,99.388177
std,577.494589,0.934133,2.924796e+09,1.505044,1.259961
min,0.000000,1.000000,0.000000e+00,17.138870,98.198715
25%,499.750000,1.000000,1.100000e+04,19.348822,99.154558
50%,999.500000,2.000000,1.800000e+04,19.383125,99.183172
75%,1499.250000,2.000000,2.800000e+04,19.432500,99.224802
max,1999.000000,16.000000,1.200000e+11,28.408360,106.864365


In [ ]:
rentas.dtypes

Unnamed: 0      int64
rooms          object
bathrooms     float64
parking          bool
precio        float64
latitude      float64
longitude     float64
date_in        object
dtype: object

In [ ]:
rentas['rooms'].unique()

array(['3', '2', 'dos', '3.0', 'uno', '2.0', 'uno y medio', '1', 'one',
       nan, '4', '1.0', 'cinco', '4.0', '6.0', '7.0', '5', '6'],
      dtype=object)

Ya que esta columna es inconsistente en sus tipos de datos, podemos un sencillo analisis estadistico de la frecuencia de estos datos. Esto en pos de poder tomar una decisión acerca de cómo tratar estos datos

In [ ]:
rentas['rooms'].value_counts(dropna=False)

dos            385
2              345
2.0            288
3.0            261
3              254
1              102
one             82
uno y medio     74
uno             61
1.0             57
4               31
4.0             29
6               10
6.0              9
NaN              6
5                3
cinco            2
7.0              1
Name: rooms, dtype: int64

In [ ]:
na_percentage = (rentas['rooms'].isna().sum() / rentas['rooms'].count()) * 100
na_percentage

0.3009027081243731

Una observación es que la columna cuenta con datos nulos. Considerando la poca representatividad de datos nulos, podrían ser removidos del dataset para reducir el ruido en la data. Se trata de una cantidad de datos nulos bastante inferior al 1% del total de registros

In [ ]:
rentas = rentas[rentas['rooms'].notna()]

In [ ]:
rentas['rooms'].isna().sum()

0

Los valores nulos de la columna rooms han sido excluidos del dataset
Para el problema de los tipos de valores inconsistentes, la frecuenia de datos con formato de string es bastante alta, bast con observar que la moda es "dos"
Esto podría ser trabajado transformando los datos tipo strings en datos tipo flotantes, así también los enteros. Para los tipo enteros bastará con parsearlos, para los strings se podría crear un diccionario para reemplazar el valor por el determinado en el diccionario.

In [ ]:
text_to_number_dict = {
    "uno" : 1.0,
    "one" : 1.0,
    "uno y medio" : 1.5,
    "dos" : 2,
    "tres" : 3,
    "cuatro" : 4,
    "cinco" : 5,
    "seis" : 6,
    "siete" : 7
}

rentas.replace({"rooms": text_to_number_dict}, inplace=True)
rentas['rooms'].value_counts(dropna=False)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


2      385
2      345
2.0    288
3.0    261
3      254
1.0    143
1      102
1.5     74
1.0     57
4       31
4.0     29
6       10
6.0      9
5        3
5        2
7.0      1
Name: rooms, dtype: int64

In [ ]:
rentas.describe()

,Unnamed: 0,bathrooms,precio,latitude,longitude
count,1994.000000,1927.000000,1.994000e+03,1994.000000,1994.000000
mean,999.888666,1.880643,1.131382e+08,19.543453,99.385376
std,577.321904,0.935220,2.929188e+09,1.494141,1.250382
min,0.000000,1.000000,0.000000e+00,17.138870,98.198715
25%,500.250000,1.000000,1.100000e+04,19.348842,99.154553
50%,999.500000,2.000000,1.800000e+04,19.383300,99.183172
75%,1499.750000,2.000000,2.800000e+04,19.432493,99.224800
max,1999.000000,16.000000,1.200000e+11,28.408360,106.864365


In [ ]:
rentas['bathrooms'].value_counts(dropna=False)

2.0     873
1.0     714
3.0     233
4.0      88
NaN      67
5.0      13
6.0       4
16.0      1
8.0       1
Name: bathrooms, dtype: int64

In [ ]:
rentas['bathrooms'].fillna(int( rentas['bathrooms'].median()),inplace=True )

In [ ]:
rentas['precio'].isna().sum()
print('Precios nulos:', rentas['precio'].isna().sum())
print("Latidudes nulas: ", rentas['latitude'].isna().sum())
print('Logintudes nulas:', rentas['longitude'].isna().sum())
print('fechas nulas:', rentas['date_in'].isna().sum())

Precios nulos: 0
Latidudes nulas:  0
Logintudes nulas: 0
fechas nulas: 0


In [ ]:
rentas['rooms'] = rentas['rooms'].astype(float)
rentas['rooms'] = rentas['rooms'].astype(int)
rentas.dtypes

Unnamed: 0      int64
rooms           int64
bathrooms     float64
parking          bool
precio        float64
latitude      float64
longitude     float64
date_in        object
dtype: object

Se ha limpiado el dataset de datos nulos
Los datos de las columnas son consistentes en su tipo de dato. 

Ahora se revisaréa el dataset en busca de outliers y determinar como serán tratados

In [ ]:
rentas.describe()

,Unnamed: 0,rooms,bathrooms,precio,latitude,longitude
count,1994.000000,1994.000000,1994.000000,1.994000e+03,1994.000000,1994.000000
mean,999.888666,2.178034,1.884654,1.131382e+08,19.543453,99.385376
std,577.321904,0.850547,0.919617,2.929188e+09,1.494141,1.250382
min,0.000000,1.000000,1.000000,0.000000e+00,17.138870,98.198715
25%,500.250000,2.000000,1.000000,1.100000e+04,19.348842,99.154553
50%,999.500000,2.000000,2.000000,1.800000e+04,19.383300,99.183172
75%,1499.750000,3.000000,2.000000,2.800000e+04,19.432493,99.224800
max,1999.000000,7.000000,16.000000,1.200000e+11,28.408360,106.864365


Observando los minimos y maximos arrojados por el query anterior, las columnas candidatas a exploración por outliers son "precio" y "bathrooms"

In [ ]:
rentas.loc[rentas['bathrooms'] == 16]

,Unnamed: 0,rooms,bathrooms,parking,precio,latitude,longitude,date_in
399,399,3,16.0,True,230000.0,19.40572,99.231834,2020-03-05 03:57:18.683360


In [ ]:
rentas.describe().apply(lambda s: s.apply('{0:.2f}'.format))

,Unnamed: 0,rooms,bathrooms,precio,latitude,longitude
count,1994.00,1994.00,1994.00,1994.00,1994.00,1994.00
mean,999.89,2.18,1.88,113138212.82,19.54,99.39
std,577.32,0.85,0.92,2929188338.63,1.49,1.25
min,0.00,1.00,1.00,0.00,17.14,98.20
25%,500.25,2.00,1.00,11000.00,19.35,99.15
50%,999.50,2.00,2.00,18000.00,19.38,99.18
75%,1499.75,3.00,2.00,28000.00,19.43,99.22
max,1999.00,7.00,16.00,120000004000.00,28.41,106.86


Se puede observar que en la columna de precios se encuentran valores atipicos en los minimos y maximos

In [ ]:
print('mediana: ', rentas['precio'].median())
print('media: ', rentas['precio'].mean())

mediana:  18000.0
media:  113138212.82074723


Comparando la media con la mediana, se puede observar una diferencia muy marcada, los outliers están afectando de manera notoria los indicadores estadisticos del dataset. Por convencion estadistica los outiliers son aquellos que se distancian 1.5 veces del rango interior entre Q3 y Q1, por lo tanto se eliminan

In [ ]:


Q1 = rentas['precio'].quantile(0.25)
Q3 = rentas['precio'].quantile(0.75)
internal_range = Q3 - Q1

rentas = rentas[~((rentas['precio'] < (Q1 - 1.5 * internal_range)) | (rentas['precio'] > (Q3 + 1.5 * internal_range)))]

rentas.describe().apply(lambda s: s.apply('{0:.2f}'.format))

,Unnamed: 0,rooms,bathrooms,precio,latitude,longitude
count,1826.00,1826.00,1826.00,1826.00,1826.00,1826.00
mean,1001.59,2.17,1.77,18583.21,19.56,99.41
std,578.61,0.85,0.76,10836.70,1.56,1.30
min,0.00,1.00,1.00,0.00,17.14,98.20
25%,499.25,2.00,1.00,10500.00,19.35,99.15
50%,1003.50,2.00,2.00,16500.00,19.38,99.18
75%,1500.75,3.00,2.00,25000.00,19.43,99.22
max,1999.00,7.00,6.00,53000.00,28.41,106.86


In [ ]:
rentas.loc[rentas['precio'] <= 3000]

,Unnamed: 0,rooms,bathrooms,parking,precio,latitude,longitude,date_in
0,0,3,1.0,True,1800.0,19.362000,99.225200,2020-06-30 03:40:29.302309
131,131,2,1.0,False,3000.0,28.408110,106.863655,2020-08-01 20:49:44.101356
151,151,5,1.0,False,3000.0,19.395834,99.096650,2020-11-16 07:56:19.405614
179,179,1,1.0,False,2900.0,19.460180,99.154490,2020-01-20 02:39:13.608192
190,190,2,1.0,False,0.0,19.442875,99.186584,2020-03-01 02:47:23.596794
191,191,3,2.0,False,3000.0,19.343800,99.051500,2020-04-28 03:27:50.294101
192,192,2,2.0,True,2300.0,19.343700,99.051600,2020-04-30 02:39:45.797910
222,222,2,1.0,True,1.0,19.537212,99.139700,2020-03-13 02:38:57.252865
403,403,2,1.0,True,2400.0,19.343500,99.051800,2020-01-31 02:51:28.637470
405,405,1,1.0,False,2200.0,19.395900,99.096600,2020-03-25 02:47:55.716350


In [ ]:
rentas = rentas[rentas['precio']>=1000]

In [ ]:
rentas.describe().apply(lambda s: s.apply('{0:.2f}'.format))

,Unnamed: 0,rooms,bathrooms,precio,latitude,longitude
count,1820.00,1820.00,1820.00,1820.00,1820.00,1820.00
mean,1001.47,2.17,1.77,18644.12,19.56,99.41
std,578.22,0.85,0.76,10802.39,1.56,1.30
min,0.00,1.00,1.00,1450.00,17.14,98.20
25%,499.75,2.00,1.00,10500.00,19.35,99.15
50%,1003.50,2.00,2.00,16540.00,19.38,99.18
75%,1499.25,3.00,2.00,25000.00,19.43,99.22
max,1999.00,7.00,6.00,53000.00,28.41,106.86


Se tomó la decisión de eliminar los valores menores a 4 digito para la columna "precio", ya que aunque estadisticamente no son clasificados como outliers, a criterio personal on incongruentes y ensucial el dataset, al tratarse de datos irreales y representar un procentaje muy pequeño de la muestra.

Ahora se renombraran las columnas para ser mas consistente en el idioma e indexar

In [ ]:
new_names = {
    'Unnamed: 0':'id',
    'rooms': 'habitaciones',
    'bathrooms' : 'baños',
    'latitude': 'latitud',
    'longitude': 'longitud',
    'parking': 'estacionamiento',
    'date_in': 'fecha_registro'
}
rentas.rename(columns = new_names, inplace = True)



/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
rentas.columns

Index(['id', 'habitaciones', 'baños', 'estacionamiento', 'precio', 'latitud',
       'longitud', 'fecha_registro'],
      dtype='object')

In [ ]:
rentas['longitud'] = rentas['longitud'] * -1
rentas.describe()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,id,habitaciones,baños,precio,latitud,longitud
count,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000
mean,1001.465934,2.170879,1.774176,18644.123187,19.558399,-99.413461
std,578.223408,0.849233,0.759107,10802.390239,1.563009,1.301504
min,0.000000,1.000000,1.000000,1450.000000,17.138870,-106.864365
25%,499.750000,2.000000,1.000000,10500.000000,19.347085,-99.224800
50%,1003.500000,2.000000,2.000000,16540.000000,19.381343,-99.179883
75%,1499.250000,3.000000,2.000000,25000.000000,19.432400,-99.154400
max,1999.000000,7.000000,6.000000,53000.000000,28.408360,-98.198720


El dataset ha sido limpiado de valores nulos, outliers, formatos inconsistentes, columnas renombradas y se configuraron lo indices del dataset. 
Se importa el segundo dataset y se comienza con su exploración

In [ ]:
vecindarios = pd.read_csv("neighbourhoods.csv", sep = ",")
vecindarios.head(10)

,ss,id,geom,nombre
0,0,0,0106000020E610000001000000010300000001000000A6...,LOMAS DE CHAPULTEPEC
1,1,1,0106000020E6100000010000000103000000010000005C...,LOMAS DE REFORMA (LOMAS DE CHAPULTEPEC)
2,2,2,0106000020E61000000100000001030000000100000028...,DEL BOSQUE (POLANCO)
3,3,3,0106000020E61000000100000001030000000100000051...,PEDREGAL DE SANTA URSULA I
4,4,4,0106000020E61000000100000001030000000100000050...,AJUSCO I
5,5,5,0106000020E61000000100000001030000000100000010...,VISTAS DEL MAUREL (U HAB)
6,6,6,0106000020E6100000010000000103000000010000006C...,IGNACIO ZARAGOZA I
7,7,7,0106000020E61000000100000001030000000100000047...,CENTRO II
8,8,13,0106000020E6100000010000000103000000010000001C...,CAMPESTRE COYOACAN (FRACC)
9,9,11,0106000020E6100000010000000103000000010000001B...,5 DE MAYO


In [ ]:
print('Geom nulos',vecindarios['geom'].isna().sum())
print('nombre nulos',vecindarios['nombre'].isna().sum())

Geom nulos 0
nombre nulos 0


In [ ]:
vecindarios.shape

(1808, 4)

Para este dataset no se encontraron registros nulo. 
Dado que se puede observar de forma rapida que la columna "id" tiene valores que no son consecuentes, por las estadisticas descriptivas del dataframe, se infiere que la columna "Unnamed: 0" es consecuente, por lo tanto se utilizará como indice y se renombrará, mientras que la columna "id" será eliminada

In [ ]:
vecindarios.describe()

,ss,id
count,1808.000000,1808.000000
mean,903.500000,905.662611
std,522.068961,523.665702
min,0.000000,0.000000
25%,451.750000,452.750000
50%,903.500000,905.500000
75%,1355.250000,1359.250000
max,1807.000000,1811.000000


In [ ]:
vecindarios.drop(columns="id", inplace=True)
new_name = {
    'Unnamed: 0':'id',
}
vecindarios.rename(columns = new_names, inplace = True)



In [ ]:
vecindarios.columns

Index(['ss', 'geom', 'nombre'], dtype='object')

In [ ]:
!pip install geopandas
!pip install pygeos
import geopandas as gpd

     |████████████████████████████████| 2.1 MB 8.1 MB/s 


In [ ]:
geo_df = gpd.GeoDataFrame(rentas, geometry=gpd.points_from_xy(rentas.longitud, rentas.latitud))
geo_df.head(5)

,id,habitaciones,baños,estacionamiento,precio,latitud,longitud,fecha_registro,geometry
0,0,3,1.0,True,1800.0,19.362000,-99.22520,2020-06-30 03:40:29.302309,POINT (-99.22520 19.36200)
1,1,2,1.0,False,3200.0,19.343523,-99.05177,2020-09-07 21:58:26.147000,POINT (-99.05177 19.34352)
2,2,2,2.0,True,27000.0,19.364412,-99.28679,2020-10-03 09:32:28.608233,POINT (-99.28679 19.36441)
3,3,3,2.0,True,8800.0,19.364382,-99.28682,2020-12-12 08:14:50.260666,POINT (-99.28682 19.36438)
4,4,2,3.0,True,36000.0,19.225096,-98.19910,2020-12-13 08:31:08.708283,POINT (-98.19910 19.22510)


In [ ]:
geo_vecindarios = gpd.GeoDataFrame(vecindarios, geometry = gpd.GeoSeries.from_wkb(vecindarios.geom))
geo_vecindarios

,ss,geom,nombre,geometry
0,0,0106000020E610000001000000010300000001000000A6...,LOMAS DE CHAPULTEPEC,"MULTIPOLYGON (((-99.22017 19.42803, -99.22009 ..."
1,1,0106000020E6100000010000000103000000010000005C...,LOMAS DE REFORMA (LOMAS DE CHAPULTEPEC),"MULTIPOLYGON (((-99.22967 19.41406, -99.22970 ..."
2,2,0106000020E61000000100000001030000000100000028...,DEL BOSQUE (POLANCO),"MULTIPOLYGON (((-99.20821 19.43282, -99.20813 ..."
3,3,0106000020E61000000100000001030000000100000051...,PEDREGAL DE SANTA URSULA I,"MULTIPOLYGON (((-99.14587 19.31979, -99.14579 ..."
4,4,0106000020E61000000100000001030000000100000050...,AJUSCO I,"MULTIPOLYGON (((-99.15854 19.33038, -99.15785 ..."
...,...,...,...,...
1803,1803,0106000020E61000000100000001030000000100000032...,VILLA COYOACAN,"MULTIPOLYGON (((-99.16178 19.34953, -99.16000 ..."
1804,1804,0106000020E6100000010000000103000000010000001C...,LOS OLIVOS (U HAB),"MULTIPOLYGON (((-99.13328 19.31296, -99.13327 ..."
1805,1805,0106000020E61000000100000001030000000100000007...,LA VIRGEN 1170 (U HAB),"MULTIPOLYGON (((-99.10494 19.32346, -99.10439 ..."
1806,1806,0106000020E6100000010000000103000000010000003F...,AVIACION CIVIL,"MULTIPOLYGON (((-99.08101 19.41136, -99.08105 ..."


###Importante

Pasado este punto se intento hacer merge de los datasets utilizando la geometría como metodo de matcheo, sin embargo las librerías requeridas no se instalan corectamente en el Google Colab. El resto se hará únicamente en el script de python